In [1]:
import osmnx as ox

In [ ]:
#loading buildings from Södermalm in Stockholm
tags = {"building": True}
gdf = ox.geometries_from_place("Södermalm, Stockholm", tags)

In [7]:
#choosing only the element_type "way", which perhaps(!) is the correct element_type
gdf = gdf.loc[(gdf.index.get_level_values('element_type') == 'way')]

In [8]:
#according to instructions from osm nx-docs, making correct projection
gdf_proj = ox.project_gdf(gdf)

In [10]:
#using geopandas to calculate areas from the geometric polygons
gdf_proj['area'] = gdf_proj.area

In [ ]:
#keeping only buildings with level information
gdf_l = gdf_proj.dropna(subset=['building:levels'])

In [ ]:
gdf_l['height'] = gdf_l['height'].astype('float')

In [ ]:
gdf_l['building:levels'] = gdf_l['building:levels'].astype('float')

In [39]:
#using the buildings with both height and level info to get an estimate of median level height
heights = gdf_l.loc[gdf_l['height'] > 0]['height'] / gdf_l.loc[gdf_l['height'] > 0]['building:levels'] 
heights

element_type  osmid    
way           7266905      6.250000
              37046494     3.866667
              37299689     3.583333
              42672700     3.291667
              149948736    3.571429
              846741207    4.131579
              846742152    8.100000
dtype: float64

In [47]:
#using the median of heights lower than the 90% percentile (clearing out messy values)
levelheight = heights.loc[heights < heights.quantile(0.9)].median()

In [48]:
levelheight

3.725

In [ ]:
#calculating an estimate of the height for all buildings with level data
gdf_l['est_height'] = gdf_l['building:levels'] * levelheight

In [ ]:
#calculating volume of the buildings
gdf_l['vol'] = gdf_l['area'] * gdf_l['est_height']

In [53]:
#volume in m^3
gdf_l['vol']

element_type  osmid     
way           7266905        17578.065771
              17024283      283572.059517
              17024436       19805.227960
              17024460        3684.860448
              17392811      206688.286232
                                ...      
              1030387591     21680.236140
              1065362442       266.149270
              1065843681       162.154248
              1065843683       936.317764
              1121465295       115.001376
Name: vol, Length: 516, dtype: float64